In [7]:
import pandas as pd
import math
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ativos
tickers = [ "DIRR3.SA", "CURY3.SA" ,"JHSF3.SA",
           "MRVE3.SA", "EVEN3.SA",
           "TRIS3.SA", "EZTC3.SA",
           "TEND3.SA", "CYRE3.SA",
           "HBOR3.SA" , "GFSA3.SA","TCSA3.SA" ,
           "LAVV3.SA" , "PLPL3.SA" , "MTRE3.SA" , "MDNE3.SA" 
            ]
period = "10y"
interval = "1wk"

In [8]:
data_dict = {}  

for ticker in tickers:
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=True, progress=False)[['Close']]
    data_dict[ticker] = df  

subplot_titles=[ticker.replace(".SA", "") for ticker in tickers]


In [9]:
# grafico
height=800
width=800

mma = 15

fig = make_subplots(
    rows=math.ceil(len(tickers)/2), cols=2, shared_xaxes=False, 
    subplot_titles=[ticker.replace(".SA", "") for ticker in tickers],  # Remove ".SA"
    vertical_spacing=0.1
)

for i, ticker in enumerate(tickers):
    df = data_dict[ticker][('Close', ticker)]  # Reutiliza os dados já baixados
    avg = df.rolling(window=mma, min_periods=1).mean()
    
    row = (i // 2) + 1
    col = (i % 2) + 1

    # fig.add_trace(go.Candlestick(
    #     x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close,
    #     name=ticker, showlegend=False), row=row, col=col)

    fig.add_trace(go.Scatter(
        x=df.index, y=avg, mode='lines',
        line=dict(color='blue', width=1),
        name="Média", showlegend=False), row=row, col=col)

fig.update_layout(title_text=f"<b>Construção civil</b><br><sub>mm{mma} do fechamento semanal</sub>",
                  height= height, width=width)
fig.update_xaxes(rangeslider_visible=False,
    tickformat="%Y",  # Mostrar apenas o ano
    dtick="M36",  # Intervalo de meses entre os ticks
    ticklabelmode="period"  # Coloca o rótulo no início do período)
)
fig.show()


In [28]:
import requests
from io import StringIO

#pra fingir que é um browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funds = pd.read_html(StringIO(r.text), decimal=',', thousands='.')[0]
dfunds = pd.DataFrame(funds)

media_pl = dfunds[dfunds['Papel'].isin([ticker.removesuffix('.SA') for ticker in tickers])]['P/L'].mean()
print(f'\nO P/L médio das empresas é {media_pl:.2f}')
media_pl_indice = dfunds[dfunds['Papel'].isin(['CURY3' , 'CYRE3' ,  'DIRR3' ,  
                                               'EVEN3' ,  'EZTC3' ,  'GFSA3' ,  
                                               'JHSF3' ,  'LAVV3' ,  'MDNE3' ,  
                                               'MRVE3' ,  'PLPL3' ,  'TEND3' ])]['P/L'].mean()
print(f'\nO P/L médio das empresas do setor no índice amplo é {media_pl_indice:.2f}')


O P/L médio das empresas é 6.30

O P/L médio das empresas do setor no índice amplo é 6.38
